In [9]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.gcLSTM import GCLSTMModel
from memory_capacity_utils import gen_lag_data, get_mem_cap_from_model

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')


running on GPU


# Load the dataset

In [11]:
dataset = np.load("datasets/multivariate_simulation_data_2.npy")
#dataset = np.load('datasets/oasis_data.npy')
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
#dataset = np.delete(dataset,88,axis=0)

In [12]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=5, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.0001, help="Learning rate")
    parser.add_argument('--memcap_coef', type=float, default=0, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [8]:
args = get_args()

# Set up the network architecture and train-validate

**Train-validate-functions**

In [13]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/test_memcap_losses' already exists.
Direc

In [15]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, args.max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, args.max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [16]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [15]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_2.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    # Create model instance
    model = GCLSTMModel(device=device, input_weights=input_weights).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1): 
                pred = model(data[t])
                real = data[t + 1]
                # print(pred)
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            
            total_loss = mae_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                            X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/5]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.04677099737455137, TP Loss: 3.661306921392679
[Validate] MAE Loss Across Timepoints: [0.06696117 0.06803638]
[Validate] TP Loss Across Timepoints: [4.10382175 4.67362261]
[Validate] MAE of Mem Caps Across Timepoints: [1.86400893 2.07992801]
Epoch [2/20]
[Train] MAE Loss: 0.039179364510346204, TP Loss: 1.6579744890332222
[Validate] MAE Loss Across Timepoints: [0.05038001 0.05493538]
[Validate] TP Loss Across Timepoints: [1.63474607 2.31778955]
[Validate] MAE of Mem Caps Across Timepoints: [1.08201098 1.68635282]
Epoch [3/20]
[Train] MAE Loss: 0.030160663460264914, TP Loss: 0.8307209035847336
[Validate] MAE Loss Across Timepoints: [0.03890918 0.04042028]
[Validate] TP Loss Across Timepoints: [0.99292862 1.22154868]
[Validate] MAE of Mem Caps Across Timepoints: [1.06761541 1.11660334]
Epoch [4/20]
[Train] MAE Loss: 0.02606405963888392, TP Loss: 0.6347104308661073
[Valid

[Validate] MAE Loss Across Timepoints: [0.03710365 0.03908427]
[Validate] TP Loss Across Timepoints: [0.99365693 1.24163198]
[Validate] MAE of Mem Caps Across Timepoints: [1.06298135 1.10988197]
Epoch [10/20]
[Train] MAE Loss: 0.026013816022896208, TP Loss: 0.669226398691535
[Validate] MAE Loss Across Timepoints: [0.03701044 0.03898271]
[Validate] TP Loss Across Timepoints: [0.99457103 1.24458945]
[Validate] MAE of Mem Caps Across Timepoints: [1.03559453 1.08486274]
Epoch [11/20]
[Train] MAE Loss: 0.025957643942092545, TP Loss: 0.6707138873171061
[Validate] MAE Loss Across Timepoints: [0.03696469 0.03897787]
[Validate] TP Loss Across Timepoints: [1.00337148 1.2603898 ]
[Validate] MAE of Mem Caps Across Timepoints: [1.00713492 1.07903757]
Epoch [12/20]
[Train] MAE Loss: 0.02584619290428236, TP Loss: 0.6607519760727882
[Validate] MAE Loss Across Timepoints: [0.03688768 0.03881317]
[Validate] TP Loss Across Timepoints: [0.99214005 1.24105048]
[Validate] MAE of Mem Caps Across Timepoints: 

[Train] MAE Loss: 0.02320525111281313, TP Loss: 0.5707033798098564
[Validate] MAE Loss Across Timepoints: [0.03453543 0.03576404]
[Validate] TP Loss Across Timepoints: [0.88757187 1.1319859 ]
[Validate] MAE of Mem Caps Across Timepoints: [1.01984783 1.09699812]
Epoch [19/20]
[Train] MAE Loss: 0.023194420398795046, TP Loss: 0.5661744906101376
[Validate] MAE Loss Across Timepoints: [0.03451803 0.03578267]
[Validate] TP Loss Across Timepoints: [0.88841575 1.13621879]
[Validate] MAE of Mem Caps Across Timepoints: [1.0397661  1.10915617]
Epoch [20/20]
[Train] MAE Loss: 0.023152149427914992, TP Loss: 0.5649141110479832
[Validate] MAE Loss Across Timepoints: [0.03446662 0.03568168]
[Validate] TP Loss Across Timepoints: [0.87983239 1.11986578]
[Validate] MAE of Mem Caps Across Timepoints: [1.05299652 1.12509311]

epochs finished with time:471.75124287605286

Current memory usage: 2697.98 MB

[Test] MAE Loss Across Timepoints: [0.032822   0.03529743]
[Test] TP Loss Across Timepoints: [0.7919394

[Validate] MAE Loss Across Timepoints: [0.03443354 0.03641258]
[Validate] TP Loss Across Timepoints: [0.80787843 1.08545804]
[Validate] MAE of Mem Caps Across Timepoints: [1.02290818 1.12831669]
Epoch [6/20]
[Train] MAE Loss: 0.02444961731089279, TP Loss: 0.5951605634763837
[Validate] MAE Loss Across Timepoints: [0.03428371 0.03623625]
[Validate] TP Loss Across Timepoints: [0.79576814 1.0584172 ]
[Validate] MAE of Mem Caps Across Timepoints: [1.02570556 1.14191045]
Epoch [7/20]
[Train] MAE Loss: 0.024262270366307348, TP Loss: 0.5920110042206943
[Validate] MAE Loss Across Timepoints: [0.03414367 0.03618566]
[Validate] TP Loss Across Timepoints: [0.7976017  1.06375659]
[Validate] MAE of Mem Caps Across Timepoints: [1.01403437 1.11614744]
Epoch [8/20]
[Train] MAE Loss: 0.024099015616229735, TP Loss: 0.5846646104473621
[Validate] MAE Loss Across Timepoints: [0.03402381 0.0361252 ]
[Validate] TP Loss Across Timepoints: [0.79793638 1.0633204 ]
[Validate] MAE of Mem Caps Across Timepoints: [0